Представь, что ты – системный-аналитик на проекте в Тинькофф. В бизнес-линии появилась задача провести совместную акцию с крупным ретейлером «Рога и Копыта». Ваша команда отвечает за интеграционное взаимодействие и надо, чтобы каждый день ретейлер отправлял в Тинькофф информацию о клиентах, учувствовавших в акции за этот день. В этой информации должны присутствовать:

·      ФИО

·      Номер телефона

·      Email

·      Список покупок в рамках акции с ценой товаров

У владельца продукта есть определённые сомнение, в том, что эта задача вообще реализуема и он попросил тебя её поисследовать. Тебе надо:

1. Понять, нет ли каких-то подводных камней в этой задаче, из-за которых она может оказаться сложнее, чем  кажется на первый взгляд.

2. Подготовить спецификацию (желательно в формате OpenAPI/swagger) одного endpoint’а, который будет принимать эту информацию на стороне Тинькофф.

3. Предложить структуру хранения этой информации в базе данных.

## 1. Понять, нет ли каких-то подводных камней в этой задаче, из-за которых она может оказаться сложнее, чем кажется на первый взгляд.

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame(columns = ['ФИО', 'Номер телефона', 'e-mail', 'Список покупок с ценой товаров'])
df.loc[1] = ['Гузенко Н.А.', '87778797286', 'nikitaguzenko2@gmail.com', 'Молоко: 60, свинина: 400, хлеб: 15']
df.loc[2] = ['Галеханов А.К.', '87777138422', 'aldiyar.2004@mail.ru', 'Спаржа: 140, гречка: 160, говядина: 500']
df.loc[3] = ['Акмурзанов А._.', '87779987907', 'almas.0711@gmail.com', 'Сметана: 75, молоко: 60, баранина: 450']
display(df)

,ФИО,Номер телефона,e-mail,Список покупок с ценой товаров
1,Гузенко Н.А.,87778797286,nikitaguzenko2@gmail.com,"Молоко: 60, свинина: 400, хлеб: 15"
2,Галеханов А.К.,87777138422,aldiyar.2004@mail.ru,"Спаржа: 140, гречка: 160, говядина: 500"
3,Акмурзанов А._.,87779987907,almas.0711@gmail.com,"Сметана: 75, молоко: 60, баранина: 450"


### У нас есть таблица полностью описывающая желание заказчика. Чисто технически данная таблица не является проблемной и создать ее не является проблемой, но видно, что поле "Список покупок с ценой товаров" очень проблемное. Дальше, когда записей станет еще больше - мы не сможем их проанализировать.

### Во-первых, каждая запись является уникальной и посчитать, даже, общее количество какого-либо продукта придется вручную. Поэтому нужно будет вносить каждый продукт новой записью.
### Во-вторых, цену товара нужно вносить отдельным полем, из-за того, что над ней нельзя произвести какие-либо вычисления. 

### Приняв все во внимание получим:

In [14]:
import pandas as pd
import numpy as np
df = pd.DataFrame(columns = ['ФИО', 'Номер телефона', 'e-mail', 'Покупка', 'Цена'])
df.loc[1] = ['Гузенко Н.А.', '87778797286', 'nikitaguzenko2@gmail.com', 'Молоко', 60]
df.loc[2] = ['Гузенко Н.А.', '87778797286', 'nikitaguzenko2@gmail.com', 'Свинина', 400]
df.loc[3] = ['Гузенко Н.А.', '87778797286', 'nikitaguzenko2@gmail.com', 'Хлеб',  15]
df.loc[4] = ['Галеханов А.К.', '87777138422', 'aldiyar.2004@mail.ru', 'Гречка', 160]
df.loc[5] = ['Галеханов А.К.', '87777138422', 'aldiyar.2004@mail.ru', 'Говядина', 500]
df.loc[6] = ['Акмурзанов А._.', '87779987907', 'almas.0711@gmail.com', 'Сметана', 75]
df.loc[7] = ['Акмурзанов А._.', '87779987907', 'almas.0711@gmail.com', 'Молоко', 60]
display(df)

,ФИО,Номер телефона,e-mail,Покупка,Цена
1,Гузенко Н.А.,87778797286,nikitaguzenko2@gmail.com,Молоко,60
2,Гузенко Н.А.,87778797286,nikitaguzenko2@gmail.com,Свинина,400
3,Гузенко Н.А.,87778797286,nikitaguzenko2@gmail.com,Хлеб,15
4,Галеханов А.К.,87777138422,aldiyar.2004@mail.ru,Гречка,160
5,Галеханов А.К.,87777138422,aldiyar.2004@mail.ru,Говядина,500
6,Акмурзанов А._.,87779987907,almas.0711@gmail.com,Сметана,75
7,Акмурзанов А._.,87779987907,almas.0711@gmail.com,Молоко,60


## Как результат, появилось много записей об одном и том же пользователе, но с такой структурой уже можно будет анализировать, считать продукты, и цены.

## Подготовить спецификацию (желательно в формате OpenAPI/swagger) одного endpoint’а, который будет принимать эту информацию на стороне Тинькофф.

openapi: 3.0.0
info:
  version: 1.0.0
  title: Тинькофф API для информации о пользователе
  description: API для получения и изменения информации о пользователе на стороне Тинькофф
servers:
  - url: https://api.tinkoff.ru
paths:
  /users/{user_id}:
    parameters:
      - name: user_id
        in: path
        description: Идентификатор пользователя
        required: true
        schema:
          type: integer
          minimum: 1
        example: 1234
    get:
      summary: Получение информации о пользователе
      description: Возвращает информацию о пользователе с указанным идентификатором
      responses:
        '200':
          description: Успешный ответ
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/User'
        '400':
          description: Некорректный запрос
        '401':
          description: Неавторизованный запрос
        '404':
          description: Пользователь не найден
        '500':
          description: Внутренняя ошибка сервера
    post:
      summary: Отправка данных о пользователе
      description: Отправляет данные о пользователе на сторону Тинькофф
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/UserData'
      responses:
        '200':
          description: Успешный ответ
        '400':
          description: Некорректный запрос
        '401':
          description: Неавторизованный запрос
        '500':
          description: Внутренняя ошибка сервера
components:
  schemas:
    User:
      type: object
      properties:
        id:
          type: integer
          description: Идентификатор пользователя
          example: 1234
        name:
          type: string
          description: ФИО пользователя
          example: Ткаченко И.В.
        email:
          type: string
          format: email
          description: Email пользователя
          example: tkach@mail.ru
        phone:
          type: string
          description: Номер телефона пользователя
          example: +79123456789
        purchases:
          type: string
          description: Покупка пользователя
          example: Milk
        price:
          type: integer
          description: Цена покупки
          example: 150
    UserData:
      type: object
      properties:
        name:
          type: string
          description: ФИО пользователя
          example: Ткаченко И.В.
        email:
          type: string
          format: email
          description: Email пользователя
          example: tkach@mail.ru
        phone:
          type: string
          description: Номер телефона пользователя
          example: +79123456789
        purchases:
          type: string
          description: Покупка пользователя
          example: Milk
        price:
          type: integer
          description: Цена покупки
          example: 150

## 3. Предложить структуру хранения этой информации в базе данных.

### SQL структура:

In [ ]:
    name VARCHAR(30),
    phone_number VARCHAR(15),
    e_mail VARCHAR(30),
    purchases VARCHAR(15),
    price INT